#### 1 Introduction

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Base URL para los chunks
base_url = "https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_"

In [3]:
# Lista para almacenar los datos de todos los chunks
all_data = []

In [4]:
# Iterar sobre las 10 páginas
for i in range(1, 11):
    # Construir la URL para cada chunk
    url = f"{base_url}{i}.html"
    print(f"Scraping {url}...")  # Opcional: para monitorear el progreso
    
    # Hacer la solicitud HTTP
    response = requests.get(url)
    
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar la tabla
    table = soup.find('table')
    
    # Extraer los encabezados
    headers = [header.text.strip() for header in table.find_all('th')]
    
    # Extraer las filas de la tabla
    rows = []
    for row in table.find_all('tr'):
        cells = [cell.text.strip() for cell in row.find_all('td')]
        if cells:
            rows.append(cells)
    
    # Crear un DataFrame temporal
    df = pd.DataFrame(rows, columns=headers)
    
    # Agregar el DataFrame temporal a la lista de todos los datos
    all_data.append(df)

Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_1.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_2.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_3.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_4.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_5.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_6.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_7.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_8.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_9.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_10.html...


In [5]:
# Combinar todos los DataFrames en uno solo
final_data = pd.concat(all_data, ignore_index=True)

In [6]:
display(final_data)

,,directorio,secuencia_p,orden,clase,dominio,mes,estrato1,sex,age,...,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
0,1,4514331,1,2,1,BOGOTA,1,2,0,29,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,2,4514331,1,1,1,BOGOTA,1,2,1,36,...,NA,NA,NA,8404.3203125,NA,NA,NA,NA,1620833.25,8404.3203125
2,3,4514332,1,4,1,BOGOTA,1,2,1,4,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,4,4514332,1,3,1,BOGOTA,1,2,1,7,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,5,4514332,1,1,1,BOGOTA,1,2,0,32,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32172,3213,4804454,1,2,1,BOGOTA,12,2,0,24,...,NA,NA,NA,3345.5556640625,NA,NA,NA,NA,1003666.6875,3345.5556640625
32173,3214,4804455,1,1,1,BOGOTA,12,3,0,36,...,5e+05,NA,NA,25958.333984375,NA,NA,NA,NA,6675000,25958.333984375
32174,3215,4804455,1,2,1,BOGOTA,12,3,1,41,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
32175,3216,4804455,1,3,1,BOGOTA,12,3,1,8,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### 1.1 General Instructions

In [ ]:
#Introduction

